In [3]:
from bs4 import BeautifulSoup
import requests
# import time, os
import re
# import urllib
# import json
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import datetime
import numpy as np

In [4]:
# ipython autotime
%load_ext autotime

# ipython_memory_usage
import ipython_memory_usage 
%ipython_memory_usage_start   

'memory profile enabled'

time: 141 ms
In [4] used 0.0625 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 103.47 MiB


In [24]:
start, end = 2400000, 2400800



time: 934 µs
In [24] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 146.47 MiB


In [25]:
allf = []
for id in range(start, end):
    site = 'https://get61.com/hand_detail?handid={}'
    page = requests.get(site.format(id)).text
    soup = BeautifulSoup(page, 'html5lib')

    cardre = re.compile('var play11')
    cards = soup.find(text=cardre)
    a = cards.split(';')
    a
    b = a[0].split(' ')
    rounds = [i.split(' ') for i in a]
    if rounds[41][-1] == '"NA"':
        continue
    else:

        plays = [rounds[i][3].replace('"', '') for i in range(30)]

        handa = plays[:6]
        handb = plays[6:12]
        handc = plays[12:18]
        handd = plays[18:24]
        hande = plays[24:32]

        nameslist = [row.split('=') for row in a[36:41]]
        scores = [int(rounds[i][3].replace('"', '')) for i in range(41, 46)]
        names = [nameslist[row][1].strip().replace('"', '') for row in range(5)]
        leads = [int(rounds[i][3].replace('"', '')) for i in range(30, 36)]

        blind1, blind2 = rounds[48][-1].replace('"',''), rounds[49][-1].replace('"','')
        bury1, bury2 = rounds[46][-1].replace('"',''), rounds[47][-1].replace('"','')

        call_ace = rounds[50][-1].replace('"','')
        ace_suit = rounds[53][-1].replace('"','')
        is_leaster = rounds[51][-1].replace('"','')

        player1 = [names[0]] + handa + [scores[0]]
        player2 = [names[1]] + handb + [scores[1]]
        player3 = [names[2]] + handc + [scores[2]]
        player4 = [names[3]] + handd + [scores[3]]
        player5 = [names[4]] + hande + [scores[4]]
        df = pd.DataFrame([player1, player2, player3, player4, player5], columns = ['name', 1, 2,3, 4, 5, 6, 'score'])
        
        df['handid'] = id
        df['picker'] = df['score'].apply(lambda x: 'picker' if x == max(scores, key=abs) else 'partner' if np.sign(x) == np.sign(max(scores,                                               key=abs)) else 'no')
        df['alone'] = df['picker'].apply(lambda x: 'no' if 'partner' in df.values else 'yes')
        blind1, blind2 = rounds[48][-1].replace('"',''), rounds[49][-1].replace('"','')
        df['position'] = [0, 1, 2, 3, 4]
        df['lead0'] = leads[0]
        df['lead1'] = leads[1]
        df['lead2'] = leads[2]
        df['lead3'] = leads[3]
        df['lead4'] = leads[4]
        df['lead5'] = leads[5]
        df['pick_position'] = df.apply(lambda x: 0 if x['position'] == x['lead0'] else x['position'] + (5 % x['lead0']) if x['position'] < x['lead0'] else x['position'] - x['lead0'], axis=1)
        df['rd0pos'] = df.apply(lambda x: 0 if x['position'] == x['lead0'] else 4 if x['lead0'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead0']) if x['position'] < x['lead0'] else x['position'] - x['lead0'], axis=1)
        df['rd1pos'] = df.apply(lambda x: 0 if x['position'] == x['lead1'] else 4 if x['lead1'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead1']) if x['position'] < x['lead1'] else x['position'] - x['lead1'], axis=1)
        df['rd2pos'] = df.apply(lambda x: 0 if x['position'] == x['lead2'] else 4 if x['lead2'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead2']) if x['position'] < x['lead2'] else x['position'] - x['lead2'], axis=1)
        df['rd3pos'] = df.apply(lambda x: 0 if x['position'] == x['lead3'] else 4 if x['lead3'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead3']) if x['position'] < x['lead3'] else x['position'] - x['lead3'], axis=1)
        df['rd4pos'] = df.apply(lambda x: 0 if x['position'] == x['lead4'] else 4 if x['lead4'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead4']) if x['position'] < x['lead4'] else x['position'] - x['lead4'], axis=1)
        df['rd5pos'] = df.apply(lambda x: 0 if x['position'] == x['lead5'] else 4 if x['lead5'] == 1 and x['position'] == 0 else x['position'] + (5 % x['lead5']) if x['position'] < x['lead5'] else x['position'] - x['lead5'], axis=1)

        df['blind_1'] = blind1
        df['blind_2'] = blind2

        df['bury1'] = bury1
        df['bury2'] = bury2

        df['call_ace'] = call_ace
        df['ace_suit'] = ace_suit
        df['leaster'] = is_leaster
        allf.append(df)

In [26]:
dftot = pd.concat(allf)

time: 522 ms
In [26] used 7.4883 MiB RAM in 0.63s, peaked 0.00 MiB above current, total RAM usage 153.96 MiB


In [27]:
dftot

,name,1,2,3,4,5,6,score,handid,picker,...,rd3pos,rd4pos,rd5pos,blind_1,blind_2,bury1,bury2,call_ace,ace_suit,leaster
0,Robert Radtke,14D,9H,13D,9D,10H,13H,-4,2400001,no,...,2,2,2,11C,7S,8C,14S,,,
1,hulkster53,8S,8H,7D,9S,13C,14C,-4,2400001,no,...,3,3,3,11C,7S,8C,14S,,,
2,upnorth46,10S,7H,12D,12H,9C,14H,-4,2400001,no,...,4,4,4,11C,7S,8C,14S,,,
3,RecordHolder37,7S,11C,12S,12C,11H,11D,16,2400001,picker,...,0,0,0,11C,7S,8C,14S,,,
4,geo4mad,13S,7C,8D,11S,10D,10C,-4,2400001,no,...,1,1,1,11C,7S,8C,14S,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Artemas16,9H,8D,13D,10S,7S,11H,-2,2400799,no,...,1,4,1,7H,12C,7H,10C,,,
1,Pat Hoffman,7C,11D,11C,12S,8H,9C,2,2400799,partner,...,2,0,2,7H,12C,7H,10C,,,
2,Baernutz,14C,9D,10D,9S,11S,14S,-2,2400799,no,...,3,1,0,7H,12C,7H,10C,,,
3,Bert Simpson,13C,8C,13S,13H,14H,10H,-2,2400799,no,...,4,2,1,7H,12C,7H,10C,,,


time: 36 ms
In [27] used 0.9297 MiB RAM in 0.14s, peaked 0.00 MiB above current, total RAM usage 154.89 MiB


In [28]:
#dftot.to_pickle('2268010_2268200')
#dftot.to_pickle('2268200_2268600')
#dftot.to_pickle('2268600_2270000')
#dftot.to_pickle('2270000_2272000')
#dftot.to_pickle('2272000_2274000')
#dftot.to_pickle('2274000_2277000')
#dftot.to_pickle('2277000_2280000')
#dftot.to_pickle('2280000_2282000')
#dftot.to_pickle('2282000_2284000')
#dftot.to_pickle('2284000_2285000')
#dftot.to_pickle('2285000_2287000')
#dftot.to_pickle('2293000_2296000')
#dftot.to_pickle('2296000_2397000')
dftot.to_pickle('2400000_2400800')

time: 14.3 ms
In [28] used 3.2070 MiB RAM in 0.12s, peaked 0.00 MiB above current, total RAM usage 158.09 MiB


In [28]:
#dftot.to_pickle('2256006_2256009')

time: 2.8 ms
In [28] used 0.0039 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 131.70 MiB


In [5]:
df1 = pd.read_pickle('2296000_2330000')
df2 = pd.read_pickle('2296000_2397000')

time: 388 ms
In [5] used 76.8359 MiB RAM in 0.49s, peaked 0.00 MiB above current, total RAM usage 180.30 MiB


In [30]:
df1

,name,1,2,3,4,5,6,score,handid,picker,...,rd3pos,rd4pos,rd5pos,blind_1,blind_2,bury1,bury2,call_ace,ace_suit,leaster
0,jturba,7C,7S,10H,13S,11D,12H,1,2296000,partner,...,1,4,2,10S,8S,13C,13H,,,
1,swood70,9C,13D,8H,12S,7D,8D,-1,2296000,no,...,2,0,3,10S,8S,13C,13H,,,
2,Nancy Ortlieb,10C,14D,14C,7H,9D,9H,-1,2296000,no,...,3,1,4,10S,8S,13C,13H,,,
3,barry g,8C,9S,14H,11H,11C,12D,-1,2296000,no,...,4,2,0,10S,8S,13C,13H,,,
4,I Will Crack You,10D,10S,12C,14S,11S,8S,2,2296000,picker,...,0,3,1,10S,8S,13C,13H,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,rboegel,8D,14S,14D,7C,11C,8H,-2,2304498,no,...,2,1,0,10C,13C,10H,10S,,,
1,gmabear,12H,9S,11D,9C,7H,9H,2,2304498,partner,...,3,2,1,10C,13C,10H,10S,,,
2,bighuhnit,12C,13S,12D,14C,8C,8S,-2,2304498,no,...,4,0,2,10C,13C,10H,10S,,,
3,Barbara Dorn,7D,13D,12S,13C,10C,10D,4,2304498,picker,...,0,1,3,10C,13C,10H,10S,,,


time: 68.7 ms
In [30] used -1.9023 MiB RAM in 0.17s, peaked 0.00 MiB above current, total RAM usage 293.01 MiB


In [31]:
df2

,name,1,2,3,4,5,6,score,handid,picker,...,rd3pos,rd4pos,rd5pos,blind_1,blind_2,bury1,bury2,call_ace,ace_suit,leaster
0,jturba,7C,7S,10H,13S,11D,12H,1,2296000,partner,...,1,4,2,10S,8S,13C,13H,,,
1,swood70,9C,13D,8H,12S,7D,8D,-1,2296000,no,...,2,0,3,10S,8S,13C,13H,,,
2,Nancy Ortlieb,10C,14D,14C,7H,9D,9H,-1,2296000,no,...,3,1,4,10S,8S,13C,13H,,,
3,barry g,8C,9S,14H,11H,11C,12D,-1,2296000,no,...,4,2,0,10S,8S,13C,13H,,,
4,I Will Crack You,10D,10S,12C,14S,11S,8S,2,2296000,picker,...,0,3,1,10S,8S,13C,13H,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,rboegel,8D,14S,14D,7C,11C,8H,-2,2304498,no,...,2,1,0,10C,13C,10H,10S,,,
1,gmabear,12H,9S,11D,9C,7H,9H,2,2304498,partner,...,3,2,1,10C,13C,10H,10S,,,
2,bighuhnit,12C,13S,12D,14C,8C,8S,-2,2304498,no,...,4,0,2,10C,13C,10H,10S,,,
3,Barbara Dorn,7D,13D,12S,13C,10C,10D,4,2304498,picker,...,0,1,3,10C,13C,10H,10S,,,


time: 60.8 ms
In [31] used 0.0664 MiB RAM in 0.17s, peaked 0.00 MiB above current, total RAM usage 293.07 MiB


In [7]:
mask = (df1['score'] < 0) & (df1['picker'] == 'picker')

time: 5.41 ms
In [7] used -1.5117 MiB RAM in 0.11s, peaked 1.54 MiB above current, total RAM usage 190.90 MiB


In [8]:
df1[mask]

,name,1,2,3,4,5,6,score,handid,picker,...,rd3pos,rd4pos,rd5pos,blind_1,blind_2,bury1,bury2,call_ace,ace_suit,leaster
1,Brew,7S,12C,9S,8H,12S,14H,-8,2296008,picker,...,3,1,0,10C,9S,10C,8C,,,
2,Dwight Schrute,12C,8C,11S,12H,7H,13H,-2,2296009,picker,...,3,0,3,13C,13H,13C,7S,,,
4,picksonschitt,7C,8C,13C,11H,7D,12S,-2,2296022,picker,...,4,0,3,12S,11H,10C,8H,,,
3,HARVEY,11H,13D,13C,10H,8C,12S,-2,2296036,picker,...,2,0,2,14H,10S,14H,10S,,,
3,Bill Burtard,13D,12D,12C,7H,9H,13H,-2,2296049,picker,...,0,2,4,13H,9S,9S,13S,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,elephant,13C,13H,7D,9H,12H,9D,-8,2304481,picker,...,1,4,0,9H,13H,14C,7S,,,
1,sebastiankoller,8H,10S,9D,11H,12C,10D,-2,2304482,picker,...,1,0,0,10H,10D,10H,14S,,,
4,jturba,14H,14D,8H,12D,10D,12H,-16,2304485,picker,...,1,1,4,10D,8H,9C,8C,,,
2,quinna2525,7S,9H,14C,12C,13H,12S,-4,2304486,picker,...,0,0,3,13S,9H,13S,13C,,,


time: 53.3 ms
In [8] used 26.6641 MiB RAM in 0.16s, peaked 0.00 MiB above current, total RAM usage 217.57 MiB
